In [ ]:
import boto3, os

In [ ]:
ec2 = boto3.resource('ec2')
route53 = boto3.client('route53')

In [ ]:
default_ip_security_group_name = 'default-using-my-ip'
security_group_name = 'master-class-security-group'
security_group_description = 'security group for ec2 masterclass'

ssh_directory_name = '/Users/dougjam/.ssh'
key_pair_name = 'master-class-key-pair'
private_key_file_name = f'{ssh_directory_name}/{key_pair_name}_rsa'
public_key_file_name = f'{private_key_file_name}.pub'
ssh_config_file_name = f'{ssh_directory_name}/config'
ssh_known_hosts_file_name = f'{ssh_directory_name}/known_hosts'

domain_name = 'doug-nicholson.net'
subdomain_name = 'master-class'
full_host_name = f'{subdomain_name}.{domain_name}'

parameter_path = '/aws/service/ami-amazon-linux-latest/'
parameter_name = 'amzn2-ami-hvm-x86_64-gp2'

In [ ]:
# Delete the DNS record for the basic instance
response = route53.list_hosted_zones_by_name(
    DNSName=domain_name
)
zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.list_resource_record_sets(
    HostedZoneId=zone_id,
    StartRecordName=full_host_name,
    MaxItems='1'
)
route53_waiter = route53.get_waiter('resource_record_sets_changed')
for resource_record_set in response['ResourceRecordSets']:
    if resource_record_set['Name'] == full_host_name + '.':
        response = route53.change_resource_record_sets(
            HostedZoneId=zone_id,
            ChangeBatch={
                'Changes': [
                    {
                        'Action': 'DELETE',
                        'ResourceRecordSet': resource_record_set
                    }
                ]
            }
        )
        route53_waiter.wait(
            Id=response['ChangeInfo']['Id'][8:]
        )

In [ ]:
# Delete any instances that were started
instance_iterator = ec2.instances.filter(
    Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_pair_name
            ]
        }
    ]
)
for instance in instance_iterator:
    instance.terminate()
for instance in instance_iterator:
    instance.wait_until_terminated()

In [ ]:
# Delete the key pair for the class from AWS.
key_pair_info_iterator = ec2.key_pairs.filter(
    KeyNames=[
        key_pair_name
    ]
)
try:
    for key_pair_info in key_pair_info_iterator:
        key_pair_info.delete()
except:
    pass

In [ ]:
# Remove the key files.
for path in [private_key_file_name, public_key_file_name]:
    if os.path.isfile(path):
        os.remove(path)

In [ ]:
# Remove the host block for the class from the ssh
# config file if it is in there.
ssh_config_lines = []
skip_flag = False
if os.path.isfile(ssh_config_file_name):
    # Read the config file and remove the host block.
    with open(ssh_config_file_name, 'r') as f:
        for line in f:
            spaces_stripped_line = line.lstrip()
            lower_cased_line = spaces_stripped_line.lower()
            if lower_cased_line[0:4] == 'host':
                if spaces_stripped_line[4:].find(full_host_name) != -1:
                    skip_flag = True
                    continue
                skip_flag = False
            if not skip_flag:
                ssh_config_lines.append(line)
    # Let's remove trailing blank lines.
    for index in range(len(ssh_config_lines), 0, -1):
        if ssh_config_lines[index - 1].replace('\n', ''):
            break
        else:
            del(ssh_config_lines[index - 1])
    # Write the lines back to the file.
    with open(ssh_config_file_name, 'w') as f:
        f.writelines(ssh_config_lines)

In [ ]:
# Remove the entry for this host in the known_hosts file
if os.path.isfile(ssh_known_hosts_file_name):
    with open(ssh_known_hosts_file_name, 'r') as f:
        lines = [line for line in f if line.find(full_host_name) == -1]
    with open(ssh_known_hosts_file_name, 'w') as f:
        f.writelines(lines)

In [ ]:
# Delete the security group.
security_group_iterator = ec2.security_groups.filter(
    Filters=[
        {
            'Name': 'group-name',
            'Values': [
                security_group_name
            ]
        }
    ]
)
for security_group in security_group_iterator:
    security_group.delete()

In [ ]:
print('all cleaned up')